In [ ]:
from PIL import Image
import base64
from openai import OpenAI
from dotenv import load_dotenv
import os
import json
from collections import defaultdict
import os
import random
random.seed(42)
load_dotenv()


def resize_image(image_path, size):
    '''resize image so that the largest edge is atmost size'''
    img = Image.open(image_path)
    width, height = img.size

    if width <= size and height <= size:
        return img
    
    if width > height:
        new_width = size
        new_height = int(height * (size / width))
    else:
        new_height = size
        new_width = int(width * (size / height))
    img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
    return img


def encode_image(image_path):
    img = resize_image(image_path, 512)
    temp_name = "temp.jpg"
    img.save(temp_name)
    with open(temp_name, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


def construct_mcq(options, correct_option):
    correct_option_letter = None
    i = "a"
    mcq = ""

    for option in options:
        if option == correct_option:
            correct_option_letter = i
        mcq += f"{i}. {option}\n"
        i = chr(ord(i) + 1)

    mcq = mcq[:-1]
    return mcq, correct_option_letter

def add_row(content, data, i, with_answer=False):  

    
    content.append({
            "type": "text",
            "text": "Image "+str(i)+": "+data["question"]+"\n"+data["mcq"],
        })
    
    content.append(
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{encode_image(data["image_path"])}",
                "detail": "low"
            }
        }
    )
    if with_answer:
        content.append(
            {
                "type": "text",
                "text": "Reasoning: {}".format(data["reasoning"]),
            }
        )
        content.append(
            {
                "type": "text",
                "text": "Answer: {}".format(data["correct_option_letter"])
            }
        )
    else:
        content.append(
            {
                "type": "text",
                "text": "Reasoning: "
            }
        )
    
    return content
   


In [ ]:

FEWSHOT_JSON = "illusionVQA/comprehension/fewshot_labels.json"
FEWSHOT_IMAGE_DIR = "illusionVQA/comprehension/FEW_SHOTS/"
EVAL_JSON = "illusionVQA/comprehension/eval_labels.json"
EVAL_IMAGE_DIR = "illusionVQA/comprehension/EVAL/"

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
model_name = "gpt-4-vision-preview"

In [ ]:

import json
with open(FEWSHOT_JSON) as f:
    fewshot_dataset = json.load(f)

for data in fewshot_dataset:
    data["image_path"] = FEWSHOT_IMAGE_DIR + data["image"]
    data["mcq"], data["correct_option_letter"] = construct_mcq(data["options"], data["answer"])

In [ ]:
with open(EVAL_JSON) as f:
    eval_dataset = json.load(f)

random.shuffle(eval_dataset)

In [ ]:
category_count = defaultdict(int)

for data in eval_dataset:
    if data["image"] not in os.listdir(EVAL_IMAGE_DIR):
        print(data["image"])
        continue
    data["image_path"] = EVAL_IMAGE_DIR + data["image"]
    data["mcq"], data["correct_option_letter"] = construct_mcq(data["options"], data["answer"])
    category_count[data["category"]] += 1

print(category_count)
print(len(eval_dataset))


In [ ]:
content = [
    {
        "type": "text",
        "text": """You'll be given an image, an instruction and some choices. You have to select the correct one. Reason about the choices in the context of the question and the image. End your answer with "Answer": {letter_of_correct_choice} without the curly brackets. Here are a few examples:""",
    }
]


i = 1
for data in fewshot_dataset:
    content = add_row(content, data, i, with_answer=True)
    i += 1

content.append({
                    "type": "text",
                    "text": "Now you try it!",
                })

next_idx = i

In [ ]:
from tqdm import tqdm
ytrue = []
ypred = []

In [ ]:
import time

MAX_RETRIES = 2
for  data in tqdm(eval_dataset):

    content_t = add_row(content.copy(), data, next_idx, with_answer=False)
    retries = MAX_RETRIES
    while True:
        try:
            response = client.chat.completions.create(
                model="gpt-4-vision-preview",
                messages=[
                    {
                        "role": "user",
                        "content": content_t,
                    }
                ],
                max_tokens=512
            )
            gpt4_answer = response.choices[0].message.content.strip()
            break
        except Exception as e:
            print(e)
            retries -= 1
            time.sleep(30)
            if retries == 0:
                gpt4_answer = 'GPT4 could not answer this question.'
                print("retries exhausted")
                break
            continue
            
    answer = data["correct_option_letter"].strip()
    ytrue.append(answer)
    ypred.append(gpt4_answer)

In [ ]:
ypred = [x[-1] for x in ypred]

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from collections import Counter

print(accuracy_score(ytrue, ypred))
print(confusion_matrix(ytrue, ypred))
print(classification_report(ytrue, ypred))

print(Counter(ytrue))
print(Counter(ypred))

In [ ]:
import prettytable
from collections import defaultdict

table = prettytable.PrettyTable()
table.field_names = ["Category", "Total", "Wrong", "Accuracy"]

got_wrong_dict = defaultdict(int)

for i in range(len(ypred)):
    if ypred[i] != ytrue[i]:
        got_wrong_dict[eval_dataset[i]["category"]] += 1
    else:
        got_wrong_dict[eval_dataset[i]["category"]] += 0


for k, v in got_wrong_dict.items():
    table.add_row([k, category_count[k], v, 1 - (v/category_count[k])])


#sort by total
table.sortby = "Total"
table.reversesort = True
print(table)
        

In [ ]:
len(ypred), len(ytrue)

In [ ]:
METRIC_SAVE_DIR = "performance_metrics/"

eval_dataset_copy = eval_dataset.copy()

print(len(eval_dataset_copy))
for i,data in enumerate(eval_dataset_copy):
    if i >= len(ypred):
        break
    if ypred[i] != ytrue[i]:
        # map letter to option f
        if "BLOCK" in ypred[i]:
            data["vlm_answer"] = "BLOCK"
        else:
            # print(ypred[i])
            # print(i)
            try:
                data["vlm_answer"] = data["options"][ord(ypred[i]) - ord("a")]
            except Exception as e:
                data["vlm_answer"] = ypred[i]
                print(ypred[i])




In [ ]:

import json
with open(METRIC_SAVE_DIR + "gpt4_reasoning_results.json", "w") as f:
    json.dump(eval_dataset_copy, f, indent=4)